> Martin Fridrich 03/2022

# Initial exploratory data analysis - Olist dataset



### Goals

We strive to examine the sales part of the dataset in such a way, so we can design and implement a meaningful processing pipeline and use the results in downstream models. To meet the goal, we analyze the size + packaging (num of records, cols, dtypes, memory requirements, coverage amongst the tables) and explore a few business aspects (basic stats, outliers, trends, customer & supplier & item, demand curve).

### Document structure

### Olist data model
<img src="imgs/olist-data-model.png" width="800"/>


# Data loading

In [5]:
import pandas as pd